In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 44.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=65bfe42b3f6aa1e885c2e8268a8bc2fc64d3bfbe92dd98b74dd89966ae2d0315
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


!java -version

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import requests
file_url = "http://www.rdatasciencecases.org/Data/Airline/2007.csv.bz2"

r = requests.get(file_url, stream = True)

with open("/content/gdrive/My Drive/Colab Datasets/2007.csv.bz2", "wb") as file:
  for block in r.iter_content(chunk_size = 1024):
    if block:
      file.write(block)


file_url = "http://www.rdatasciencecases.org/Data/Airline/2008.csv.bz2"

r = requests.get(file_url, stream = True)

with open("/content/gdrive/My Drive/Colab Datasets/2008.csv.bz2", "wb") as file:
  for block in r.iter_content(chunk_size = 1024):
    if block:
      file.write(block)


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql.functions import isnan, when, count, col

In [0]:
CSV_2007= "/content/gdrive/My Drive/Colab Datasets/2007.csv.bz2" 
CSV_2008= "/content/gdrive/My Drive/Colab Datasets/2008.csv.bz2"
APP_NAME = "Flight Delays"
SPARK_URL = "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [0]:
# Connect to the Spark server

spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

# Load datasets

df_2007 = spark.read.options(header="true",inferschema = "true").csv(CSV_2007)
df_2008 = spark.read.options(header="true",inferschema = "true").csv(CSV_2008)

# We concatenate both datasets

df = df_2007.unionAll(df_2008)

In [10]:
# What's the data shape before starting cleaning ?

print(f"The shape is {df.count():d} rows by {len(df.columns):d} columns.")

The shape is 14462943 rows by 29 columns.


In [11]:
# What's the number of null values ?

null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) 
                         for c in df.columns]).toPandas().to_dict(orient='records')

print(f"We have {sum(null_counts[0].values()):d} null values in this dataset.")

We have 14248147 null values in this dataset.


In [0]:
# Drop null columns and inputs ?

df = df.drop(df.CancellationCode)
df = df.na.drop()

In [13]:
# Confirm there are no null values

null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) 
                         for c in df.columns]).toPandas().to_dict(orient='records')

print(f"We have {sum(null_counts[0].values()):d} null values in this dataset.")

We have 0 null values in this dataset.


In [14]:
# What's the data shape after cleaning ?

print(f"The shape is {df.count():d} rows by {len(df.columns):d} columns.")

The shape is 14379556 rows by 28 columns.


In [15]:
# What are the column's type ?

df.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('DepTime', 'string'),
 ('CRSDepTime', 'int'),
 ('ArrTime', 'string'),
 ('CRSArrTime', 'int'),
 ('UniqueCarrier', 'string'),
 ('FlightNum', 'int'),
 ('TailNum', 'string'),
 ('ActualElapsedTime', 'string'),
 ('CRSElapsedTime', 'string'),
 ('AirTime', 'string'),
 ('ArrDelay', 'string'),
 ('DepDelay', 'string'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('Distance', 'int'),
 ('TaxiIn', 'string'),
 ('TaxiOut', 'string'),
 ('Cancelled', 'int'),
 ('Diverted', 'int'),
 ('CarrierDelay', 'string'),
 ('WeatherDelay', 'string'),
 ('NASDelay', 'string'),
 ('SecurityDelay', 'string'),
 ('LateAircraftDelay', 'string')]

In [0]:
# Create list of feature columns

feature_cols = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 
                'CRSArrTime', 'FlightNum', 'Distance', 'Diverted']

In [0]:
# Generate and create our new feature vector column

df = VectorAssembler(inputCols=feature_cols, outputCol="features").transform(df)

In [18]:
# Select input columns

df.select("Cancelled", "features").show(5)

+---------+--------------------+
|Cancelled|            features|
+---------+--------------------+
|        0|[2007.0,1.0,1.0,1...|
|        0|[2007.0,1.0,1.0,1...|
|        0|[2007.0,1.0,1.0,1...|
|        0|[2007.0,1.0,1.0,1...|
|        0|[2007.0,1.0,1.0,1...|
+---------+--------------------+
only showing top 5 rows



In [0]:
# Build the training indexers

# Generate a labelIndexer
labelIndexer = StringIndexer(inputCol="Cancelled", outputCol="indexedLabel").fit(df)

# Generate the indexed feature vector
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)
    
# Split the data into training and tests sets
(trainingData, testData) = df.randomSplit([TRAINING_DATA_RATIO, 1 - TRAINING_DATA_RATIO])

# Train the RandomForest model
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=RF_NUM_TREES)

# Chain indexers and the forest models in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [0]:
# Train the model

model = pipeline.fit(trainingData)

In [0]:
# Make predictions

predictions = model.transform(testData)

In [22]:
# Select prediction, true label and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Error = {(1.0 - accuracy):g}")
print(f"Accuracy = {accuracy:g}")

Test Error = 0.0149549
Accuracy = 0.985045
